In [13]:
import string
import random
import pandas as pd
from typing import List
from pyspark.sql.pandas.functions import pandas_udf
from pyspark.sql.types import StringType
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline

COLUMNS = ["name","age","salary"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [2]:
def create_random_data(row_num:int)->List[tuple]:
     result = list()
     a_str = string.ascii_uppercase
     for i in range(row_num):
         random_letter = random.choice(a_str)
         result.append((random_letter,random.randint(1,row_num),random.random()))
     return result

In [3]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)
    spark_session = session_factory.build_session()
    df = spark_session.createDataFrame(data,COLUMNS).cache()
    df.toPandas().head(5)

In [4]:
session_factory_normal = DemoSQLSessionFactory(name="normal")
%time test_performance(session_factory_normal)

CPU times: user 3.18 s, sys: 55.7 ms, total: 3.24 s
Wall time: 13.1 s


In [5]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%time test_performance(session_factory_arrow)

CPU times: user 2.68 s, sys: 17.6 ms, total: 2.7 s
Wall time: 3.26 s


常规的HelloWorld的example。
页面上面的第一个例子。本质就是生成一个新的dataframe
1. 在annotation上面列出的是新的dataframe的col和类型
2. 他会自动的把pd的转换成spark的
3. 函数应该会分批node执行。然后再汇总。因为我看到了。hello world的函数会被执行好几次

In [6]:
session_factory_arrow.add_config('spark.sql.execution.arrow.maxRecordsPerBatch',10)
spark = session_factory_arrow.build_session()
test_data = create_random_data(row_num=1000)
basic_df = spark.createDataFrame(test_data,COLUMNS)
basic_df.show()

+----+---+--------------------+
|name|age|              salary|
+----+---+--------------------+
|   F|707|   0.812187771740766|
|   B|218|  0.7314196048714136|
|   D|121|  0.7969774110261034|
|   Z|167|  0.6048949931181556|
|   I|231|  0.4425588872893208|
|   F|923|   0.786461040523636|
|   E|224|  0.6008017258769238|
|   H|861|  0.3985002777129232|
|   K|265|  0.5868977369408303|
|   S|515|  0.6998075213477662|
|   U|991|  0.8532631844261254|
|   K| 62| 0.06972587357982074|
|   A|675|   0.771642388640452|
|   C|208|  0.6162263414662013|
|   R|172|  0.7694505306855899|
|   H|209| 0.24354272924336706|
|   K|213|0.015653818926125607|
|   H|156|  0.5330514618813238|
|   B|748|  0.8394659136558289|
|   P|581|    0.88864851447575|
+----+---+--------------------+
only showing top 20 rows



In [7]:
@pandas_udf("total double")
def func(s1: pd.Series, s2: pd.Series) -> pd.DataFrame:
    print("execute")
    s3 = pd.DataFrame()
    s3['total'] = s1 + s2
    return s3
basic_df.select(func("age","salary").alias("result")).show()

+--------------------+
|              result|
+--------------------+
| [707.8121877717408]|
| [218.7314196048714]|
| [121.7969774110261]|
|[167.60489499311817]|
|[231.44255888728932]|
| [923.7864610405236]|
|[224.60080172587692]|
|  [861.398500277713]|
|[265.58689773694084]|
| [515.6998075213478]|
| [991.8532631844262]|
| [62.06972587357982]|
| [675.7716423886404]|
| [208.6162263414662]|
| [172.7694505306856]|
|[209.24354272924336]|
|[213.01565381892613]|
| [156.5330514618813]|
| [748.8394659136559]|
| [581.8886485144758]|
+--------------------+
only showing top 20 rows



主要是想要看看。select方法，不能不能接受一个List

In [15]:
def to_str_func(s1: pd.Series) -> pd.Series:
    return s1.astype(dtype=str)
to_str = pandas_udf(to_str_func, returnType=StringType())

age_c = to_str("age").alias("age")
salary_c = to_str("salary").alias("salary")
selects = [age_c,salary_c]
basic_df.select(selects).show()

+---+--------------------+
|age|              salary|
+---+--------------------+
|707|   0.812187771740766|
|218|  0.7314196048714136|
|121|  0.7969774110261034|
|167|  0.6048949931181556|
|231|  0.4425588872893208|
|923|   0.786461040523636|
|224|  0.6008017258769238|
|861|  0.3985002777129232|
|265|  0.5868977369408303|
|515|  0.6998075213477662|
|991|  0.8532631844261254|
| 62| 0.06972587357982074|
|675|   0.771642388640452|
|208|  0.6162263414662013|
|172|  0.7694505306855899|
|209| 0.24354272924336706|
|213|0.015653818926125607|
|156|  0.5330514618813238|
|748|  0.8394659136558289|
|581|    0.88864851447575|
+---+--------------------+
only showing top 20 rows

